In [5]:
!sudo curl https://repo1.maven.org/maven2/com/redislabs/spark-redis_2.12/3.0.0/spark-redis_2.12-3.0.0.jar --output /usr/local/spark/jars/spark-redis_2.12-3.0.0.jar
!sudo curl https://repo1.maven.org/maven2/redis/clients/jedis/3.6.3/jedis-3.6.3.jar --output /usr/local/spark/jars/jedis-3.6.3.jar
!sudo curl https://repo1.maven.org/maven2/org/apache/commons/commons-pool2/2.9.0/commons-pool2-2.9.0.jar --output /usr/local/spark/jars/commons-pool2-2.9.0.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  271k  100  271k    0     0  2746k      0 --:--:-- --:--:-- --:--:-- 2718k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  846k  100  846k    0     0  7237k      0 --:--:-- --:--:-- --:--:-- 7237k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  131k  100  131k    0     0  1965k      0 --:--:-- --:--:-- --:--:-- 1965k


In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
#      .config("spark.redis.auth", "passwd")\
spark = SparkSession \
    .builder \
    .master("local") \
    .appName('jupyter-pyspark') \
      .config("spark.redis.host", "redis")\
      .config("spark.redis.port", "6379")\
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

21/10/25 13:18:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
grades = spark.read.csv("/home/jovyan/datasets/grades/*.tsv", inferSchema=True,sep="\t")
grades = grades.withColumnRenamed('_c0','year').withColumnRenamed("_c1","semester")\
    .withColumnRenamed("_c2","course").withColumnRenamed("_c3","credits").withColumnRenamed("_c4","grade")
grades.show()

+----+--------+------+-------+-----+
|year|semester|course|credits|grade|
+----+--------+------+-------+-----+
|2016|    Fall|IST346|      3|    A|
|2016|    Fall|CHE111|      4|   A-|
|2016|    Fall|PSY120|      3|   B+|
|2016|    Fall|IST256|      3|    A|
|2016|    Fall|ENG121|      3|   B+|
|2015|    Fall|IST101|      1|    A|
|2015|    Fall|IST195|      3|    A|
|2015|    Fall|IST233|      3|   B+|
|2015|    Fall|SOC101|      3|   A-|
|2015|    Fall|MAT221|      3|    C|
|2016|  Spring|GEO110|      3|   B+|
|2016|  Spring|MAT222|      3|    A|
|2016|  Spring|SOC121|      3|   C+|
|2016|  Spring|BIO240|      3|   B-|
|2017|  Spring|IST462|      3|    A|
|2017|  Spring|MAT411|      3|    C|
|2017|  Spring|SOC422|      3|   B-|
|2017|  Spring|ENV201|      3|   A-|
+----+--------+------+-------+-----+



In [9]:
# Save to Redis
grades.write.format("org.apache.spark.sql.redis")\
  .mode("overwrite")\
  .option("table", "grades")\
  .option("key.column","course")\
  .save()

Now on to redis-cli
```
keys "grades:*"
hgetall "grades:MAT221"
```


In [13]:
# loading back from redis, need the key column
df = spark.read.format("org.apache.spark.sql.redis")\
  .option("table", "grades")\
  .option("key.column", "course")\
  .load()
df.createOrReplaceTempView("grades")
spark.sql("select * from grades where grade = 'A'").show()

+----+--------+------+-------+-----+
|year|semester|course|credits|grade|
+----+--------+------+-------+-----+
|2016|    Fall|IST256|      3|    A|
|2015|    Fall|IST101|      1|    A|
|2016|  Spring|MAT222|      3|    A|
|2016|    Fall|IST346|      3|    A|
|2017|  Spring|IST462|      3|    A|
|2015|    Fall|IST195|      3|    A|
+----+--------+------+-------+-----+

